OK - vi skal have preprocesset det her data vi indsamler....



In [1]:
# import libraries
from pathlib import Path
import pandas as pd
import time
import os
import tqdm

In [2]:
# Replace 'your_file.csv' with the actual path to your CSV file
input_fp = Path.cwd() / 'inlier_data' # path of files to be found

output_fp = Path.cwd() / 'agg_data' # output path of files generated
# Use the Path object to actually create the subfolder
Path.mkdir(output_fp, exist_ok=True)

In [3]:
def aar(str):
    return str.split('-')[0]

In [9]:
def agg_data(df):
    
    # der er ingen grund til at tro at kolonnerne konsekvent hedder hvad de burde hedde:
    column_mapping = {
        "municipalities": "muni_code",
        "municipality": "muni_code",
        "muni_code": "muni_code",
        "municode": "muni_code",
        "square meters_price":"square_meters_price"
    }

    # Loop igennem hver DataFrame og omdøb kolonnerne, hvis de findes. trækker year ud af datoen 
    df = df\
        .rename(columns=column_mapping, inplace=False)\
        .assign(year = df["date_of_sale"].apply(lambda row: aar(row)))
    
    '''
    Hvad vil vi så gruppere de her data på? Det er der ingen der ved. Men vi må hellere gøre noget, så vi kan komme videre...
    Hvad vil vi gruppere på? Det ved vi ikke helt. Vi skal have grupperet på muni_code. Ville vi også have på housing type. Det vil vi nok.Vi skal have trukket et år ud, for det er der ikke. Antal værelser? Type of sale?
    Uanset hvad, så skal den spytte en dataframe ud vi kan gøre noget videre med. og den skal gemmes til fil.
    '''

    group_vars = ["muni_code", "housing_type", "year"]
    agg_vars = ["square_meters_price"]
    
    df = df\
        .groupby(group_vars)[agg_vars].agg(["mean", "count"])\
        .reset_index()
    
    '''
    Vi har nogen fæle outliers. Det kunne vi gøre noget ved på denne måde:
    test["kantil"] = test.groupby(group_vars)[agg_vars].transform(lambda x: x.quantile(0.8))
    test[test['square_meters_price'] <= test['kantil']].groupby("year")["square_meters_price"].hist()
    '''
    
    #af med multi index!
    df.columns = range(df.shape[1])
    df.columns = [0,1,"muni_code", "housing_type", "year", "avg_sqm_price", "count"]
    df = df[["muni_code", "housing_type", "year", "avg_sqm_price", "count"]]

    return df

In [10]:
start_time = time.time()

# Iterate through log files in the folder
for filename in tqdm.tqdm(sorted(os.listdir(input_fp))):
    if filename.startswith('inlier_sales_1992_2022_') and filename.endswith('.csv'):
        df = pd.read_csv(input_fp/filename) # read the CSV file into a DataFrame
        new_df = agg_data(df)
        
        # Rename and save the new DataFrame to a CSV file without index numbers
        output_filename = filename.replace('inlier_', 'agg_')  # Replace 'inlier_' with 'agg_' in filename
        new_df.to_csv(f'{output_fp}/{output_filename}', index=False)

elapsed_time = time.time() - start_time
print(f'Runtime: {elapsed_time}')

 33%|█████████████████████▋                                           | 1/3 [00:00<00:00, 20.73it/s]


ValueError: Length mismatch: Expected axis has 5 elements, new values have 7 elements

In [12]:
df = pd.read_csv('inlier_data/inlier_sales_1992_2022_161.csv')

In [15]:
# der er ingen grund til at tro at kolonnerne konsekvent hedder hvad de burde hedde:
column_mapping = {
    "municipalities": "muni_code",
    "municipality": "muni_code",
    "muni_code": "muni_code",
    "municode": "muni_code",
    "square meters_price":"square_meters_price"
}

# Loop igennem hver DataFrame og omdøb kolonnerne, hvis de findes. trækker year ud af datoen 
df = df\
    .rename(columns=column_mapping, inplace=False)\
    .assign(year = df["date_of_sale"].apply(lambda row: aar(row)))

In [16]:

group_vars = ["muni_code", "housing_type", "year"]
agg_vars = ["square_meters_price"]

df = df\
    .groupby(group_vars)[agg_vars].agg(["mean", "count"])\
    .reset_index()


In [18]:
df

,0,1,2,3,4
0,161,Ejerlejlighed,1992,10980.418803,117
1,161,Ejerlejlighed,1993,71789.604938,162
2,161,Ejerlejlighed,1994,6174.088235,68
3,161,Ejerlejlighed,1995,42871.463855,166
4,161,Ejerlejlighed,1996,9103.950000,100
...,...,...,...,...,...
88,161,Villa,2018,22993.475524,143
89,161,Villa,2019,23719.652174,138
90,161,Villa,2020,26677.388158,152
91,161,Villa,2021,29876.473684,133


In [17]:

#af med multi index!
df.columns = range(df.shape[1])
df.columns = [0,1,"muni_code", "housing_type", "year", "avg_sqm_price", "count"]
df = df[["muni_code", "housing_type", "year", "avg_sqm_price", "count"]]


ValueError: Length mismatch: Expected axis has 5 elements, new values have 7 elements